**import the necessary libraries and create a connection to DuckDB. also points DuckDB directly at our partitioned Parquet file.**



In [1]:
import duckdb
import pandas as pd

# The path to your partitioned Parquet directory
# The '/*/*/*.parquet' glob pattern tells DuckDB to read all files in the partitioned structure
parquet_path = '../output/olist_master.parquet/*/*/*.parquet'

# Create a connection to an in-memory DuckDB database
con = duckdb.connect(database=':memory:', read_only=False)

# Register the Parquet file as a virtual table named 'ecommerce_data'
con.execute(f"CREATE VIEW ecommerce_data AS SELECT * FROM read_parquet('{parquet_path}');")

print("Successfully connected to the Parquet data warehouse.")

Successfully connected to the Parquet data warehouse.


**run a basic query to confirm the connection works and see the table schema.**

In [2]:
# Show the first 5 rows of the virtual table
con.execute("SELECT * FROM ecommerce_data LIMIT 5;").fetchdf()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_sequential,payment_type,...,seller_zip_code_prefix,seller_city,seller_state,product_category_name_english,delivery_time_days,is_late,seller_handling_time_days,freight_ratio,month,year
0,d3c8851a6651eeff2f73b0e011ac45d0,957f8e082185574de25992dc659ebbc0,processing,2016-10-05 22:44:13,2016-10-06 15:51:05,NaT,NaT,2016-12-09,1.0,credit_card,...,69900.0,rio branco,AC,books_technical,NaN,False,NaN,0.122996,10,2016
1,cbbb524a0e1646aa6cf7a3c0bbe517ad,dacb079d55ffb1d3955c5d923df3ebb7,delivered,2016-10-05 07:31:49,2016-10-06 02:46:24,2016-10-10 02:46:24,2016-10-16 14:36:00,2016-12-07,1.0,boleto,...,30190.0,belo horizonte,MG,computers_accessories,11.294572,False,4.000000,0.274400,10,2016
2,ac2b7c522d811acba0aa270ed3e112e4,ef21aebbb093a6db29ccc6aa0b89c347,delivered,2016-10-05 15:08:00,2016-10-06 15:44:26,2016-10-10 15:44:27,2016-10-13 15:44:27,2016-11-29,1.0,credit_card,...,86035.0,londrina,PR,auto,8.025313,False,4.000012,0.068973,10,2016
3,7033745709b7cf1bac7d2533663592de,7f0ca17bb33b230b47459437cf0682c7,delivered,2016-10-04 14:13:22,2016-10-04 14:46:48,2016-10-08 14:46:49,2016-10-11 14:46:49,2016-11-30,1.0,credit_card,...,87900.0,loanda,PR,furniture_decor,7.023229,False,4.000012,0.187540,10,2016
4,5cd498954e2b37d71b315166809b4bd7,ff1a56726b7ea149c7423865609cc0c8,delivered,2016-10-07 00:54:40,2016-10-08 03:56:15,2016-10-25 11:35:52,2016-10-27 17:32:07,2016-12-01,1.0,boleto,...,17510.0,marilia,SP,perfumery,20.692674,False,17.319178,0.348107,10,2016


### Advanced Validation with Complex Queries
**Query 1: Sales by Product Category (Aggregation)**

In [3]:
# Top 10 product categories by total revenue
query1 = """
SELECT
    product_category_name_english AS category,
    COUNT(order_id) AS total_orders,
    SUM(price) AS total_revenue
FROM
    ecommerce_data
WHERE
    product_category_name_english IS NOT NULL
GROUP BY
    category
ORDER BY
    total_revenue DESC
LIMIT 10;
"""
display(con.execute(query1).fetchdf())

,category,total_orders,total_revenue
0,health_beauty,10032,1301947.97
1,watches_gifts,6213,1254322.95
2,bed_bath_table,11988,1107249.09
3,sports_leisure,9004,1029603.88
4,computers_accessories,8150,950053.69
5,furniture_decor,8832,772096.17
6,housewares,7380,668880.94
7,cool_stuff,3999,664637.13
8,auto,4400,618395.50
9,garden_tools,4590,519473.33


**Query 2: Late Deliveries vs. Review Scores (CTE and Aggregation)**

In [4]:
# Compare average review scores for on-time vs. late deliveries
query2 = """
WITH delivery_performance AS (
    SELECT
        order_id,
        review_score,
        CASE
            WHEN is_late = TRUE THEN 'Late'
            ELSE 'On-Time'
        END AS delivery_status
    FROM ecommerce_data
)
SELECT
    delivery_status,
    AVG(review_score) AS average_review_score,
    COUNT(order_id) AS number_of_orders
FROM
    delivery_performance
GROUP BY
    delivery_status;
"""
display(con.execute(query2).fetchdf())

,delivery_status,average_review_score,number_of_orders
0,Late,2.546542,9068
1,On-Time,4.134724,110075


**Query 3: Top 5 Sellers by Revenue in a Specific Year (Window Function)**

In [5]:
# Rank sellers by revenue within the year 2017
query3 = """
WITH seller_yearly_revenue AS (
    SELECT
        seller_id,
        year,
        SUM(price) as yearly_revenue,
        RANK() OVER (PARTITION BY year ORDER BY SUM(price) DESC) as revenue_rank
    FROM
        ecommerce_data
    WHERE
        year = 2017 AND seller_id IS NOT NULL
    GROUP BY
        seller_id, year
)
SELECT
    seller_id,
    yearly_revenue,
    revenue_rank
FROM
    seller_yearly_revenue
WHERE
    revenue_rank <= 5;
"""
display(con.execute(query3).fetchdf())

,seller_id,yearly_revenue,revenue_rank
0,53243585a1d6dc2643021fd1853d8905,199727.43,1
1,7e93a43ef30c4f03f38b393420bc753a,165835.43,2
2,4a3ca9315b744ce9f8e9374361493884,137164.52,3
3,46dc3b2cc0980fb8ec44634e21d2718e,112247.86,4
4,7a67c85e85bb2ce8582c35f2203ad736,104933.48,5
